In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
import scipy
from scipy import stats
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import transformers as ppb

### Preprocessing data

In [2]:
table_1 = pd.read_excel("1_Многоквартирные дома с технико-экономическими характеристиками.xlsx")
table_2 = pd.read_excel("2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства.xlsx")
table_3 = pd.read_excel("3_Работы по капитальному ремонту, проведенные в многоквартирных домах.xlsx")
table_4_1 = pd.read_excel("4_Виды работ по капитальному ремонту многоквартирных домов.xlsx")
table_4_2 = pd.read_excel("4_Виды работ по содержанию и ремонту общего имущества многоквартирных домов.xlsx")
table_5 = pd.read_excel("5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx")

table_1 = table_1.drop(labels=[0], axis=0)
table_4_1 = table_4_1.drop(labels=[0], axis=0)

In [3]:
# All date make date
#table 2
table_2["Дата создания во внешней системе"] = pd.to_datetime(table_2["Дата создания во внешней системе"], 
                                                             format = "%Y-%m-%d %H:%M:%S.%f",
                                                             errors = 'coerce')
table_2["Дата закрытия"] = pd.to_datetime(table_2["Дата создания во внешней системе"], 
                                          format = "%Y-%m-%d %H:%M:%S.%f",
                                          errors = 'coerce')
#table 3
table_3["PLAN_DATE_START"] = pd.to_datetime(table_3["PLAN_DATE_START"],
                                            #format="%Y.%m.%d",
                                            errors='coerce')
table_3["PLAN_DATE_END"] = pd.to_datetime(table_3["PLAN_DATE_END"],
                                            #format="%Y.%m.%d",
                                            errors='ignore')
table_3["FACT_DATE_START"] = pd.to_datetime(table_3["FACT_DATE_START"],
                                            #format="%Y.%m.%d",
                                            errors='ignore')
table_3["FACT_DATE_END"] = pd.to_datetime(table_3["FACT_DATE_END"],
                                            #format="%Y.%m.%d",
                                            errors='ignore')

In [4]:
# function standartization
def StandardCol(table: pd.DataFrame, name: str):
    table[name] = table[name].astype(float)
    if table_1[name].isna().sum() != 0:
        table[name] = table[name].fillna(0)
    return stats.zscore(table[name])

In [5]:
#BERT tokenizer
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

# удаление нахер вссего NaN дерьма
for count, elem in enumerate(table_2["Наименование"].isna()):
    if elem:
        table_2 = table_2.drop(labels=[count], axis=0)
table_2["Наименование"].isna().sum()

# apply tokenizator
df = table_2["Наименование"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

# find max_len
max_len = 0
for i in df.values:
    if len(i) > max_len:
        max_len = len(i)

if max_len % 2 == 1:
    max_len += 1

# add <pad> symbol
padded = np.array([i + [0]*(max_len-len(i)) for i in df.values])

### Make dataloader

In [6]:
X_batch = []
y_batch = []
class CustomDataset(Dataset):
    def __init__(self, table_1, table_2, table_3, padded = None):
        super().__init__()
        # make Bert Model for make embedding requests
        model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
        # Load tokenizer
        self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        self.features_table_1 = [ "COL_756", "COL_758", "COL_759", "COL_760", "COL_761", "COL_762", 
                              "COL_763", "COL_764", "COL_769", "COL_770", "COL_771", "COL_772", 
                              "COL_781", "COL_3363"]
        self.features_table_2 = [ "Дата закрытия", "Дата создания во внешней системе"]
        self.features_table_3 = ["WORK_NAME", "PLAN_DATE_START", "FACT_DATE_START"] # можно ещё "PLAN_DATE_START", "FACT_DATE_START"
        self.unique_unom = list(set(table_1["COL_782"].apply(int)) & set(table_2["unom"].apply(int)) & set(table_3["UNOM"].apply(int)))
        self.dict_all_works = {name: count for count,name in enumerate(set(table_3["WORK_NAME"]))}
        self.dict_all_works_zero = {name: 0 for name in set(table_3["WORK_NAME"])}
        self.table_1 = table_1
        self.table_2 = table_2
        self.table_3 = table_3
        self.padded = padded #pd.Dself.encoder_text(table_2["Наименование"])



    def __len__(self):
        return len(self.unique_unom)


    def __getitem__(self, idx):
        unom = self.unique_unom[idx]
        # idx = unom
        # get needed table

        table_1_unom = table_1.loc[(table_1["COL_782"].apply(int)==unom)]
        table_2_unom = table_2.loc[(table_2["unom"].apply(int)==unom)]
        table_3_unom = table_3.loc[(table_3["UNOM"].apply(int)==unom)]

        work_unom_time = self.get_difference_time_work(table_3_unom)

        table_3_unom['WORK_NAME'] = table_3_unom['WORK_NAME'].apply((lambda x: self.dict_all_works[x]))
        # choose needed coloumn in table
        table_1_res = self.get_table(table_1_unom, self.features_table_1).apply(float).to_numpy()
        table_2_res = self.get_table(table_2_unom, self.features_table_2)
        table_2_res = self.convert_date(table_2_res)
        #print("table_2_res", table_2_res[0][0])
        table_3_res = self.get_table(table_3_unom, self.features_table_3)
        #print("table_3_res", table_3_res)
        table_2_text = self.get_padded_sent(unom)
        #print("table_2_text", table_2_text)
        # get right result
        print(work_unom_time)
        #work_unom_time = self.get_difference_time_work(table_3_unom)
        #print(table_1_res, table_2_res, table_3_res)
        X_batch.append((table_1_res, table_2_res, table_3_res, table_2_text))
        y_batch.append(pd.DataFrame(work_unom_time).to_numpy())
        return  0,0#torch.tensor((table_1_res, table_2_res, table_3_res, table_2_text)), torch.tensor(work_unom_time)
    

    def convert_date(self, table):
        new_t = {"day_beg": [], "month_beg": [], "year_beg": [], "day_end": [], "month_end": [], "year_end": []}
        new_t["day_beg"] = table["Дата создания во внешней системе"].dt.day.to_numpy()
        new_t["month_beg"] = table["Дата создания во внешней системе"].dt.month.to_numpy()
        new_t["year_beg"]  = table["Дата создания во внешней системе"].dt.year.to_numpy()
        new_t["hour_beg"]  = table["Дата создания во внешней системе"].dt.hour.to_numpy()
        new_t["minute_beg"]  = table["Дата создания во внешней системе"].dt.minute.to_numpy()
        new_t["day_end"] = table["Дата закрытия"].dt.day.to_numpy()
        new_t["month_end"] = table["Дата закрытия"].dt.month.to_numpy()
        new_t["year_end"] = table["Дата закрытия"].dt.year.to_numpy()
        new_t["hour_end"] = table["Дата закрытия"].dt.hour.to_numpy()
        new_t["minute_end"] = table["Дата закрытия"].dt.minute.to_numpy()
        return pd.DataFrame(new_t)
    

    def get_padded_sent(self, unom: int):
        index = self.table_2[self.table_2['unom'].apply(int) == unom].index 
        pad_sent = []
        for idx in index:
            pad_sent.append(self.padded[idx])
        return np.asarray(pad_sent)


    def get_table(self, table, features):
        dict_new_table = {name: [] for name in features}
        for name in features:
            dict_new_table[name] = table[name]
        return pd.DataFrame(dict_new_table)
    

    def get_difference_time_work(self, table): # передаем данные по конкретному уному
        # get list all work
        work_unom_time = self.dict_all_works_zero
        # get date fact and plan work
        fact_beg_day = table["FACT_DATE_START"].dt.day
        fact_beg_month = table["FACT_DATE_START"].dt.month
        fact_beg_year  = table["FACT_DATE_START"].dt.year
        plan_beg_day = table["PLAN_DATE_START"].dt.day
        plan_beg_month = table["PLAN_DATE_START"].dt.month
        plan_beg_year = table["PLAN_DATE_START"].dt.year
        # compute result difference 
        result = ((plan_beg_year - fact_beg_year) * 365 + (plan_beg_month - fact_beg_month) * 30 + (plan_beg_day - fact_beg_day)).to_numpy()
        # compute mean time deviation for all works
        for count, name in enumerate(table["WORK_NAME"]):
            work_unom_time[name] =  (work_unom_time[name] + result[count]) / 2
        values = []
        for key in work_unom_time.keys():
            values.append(work_unom_time[key])
        return values
    

    def encoder_text(self, coloumn_text):
        # apply tokenizer
        with torch.no_grad():
            coloumn = coloumn_text.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
        # <pad>
        max_len = 0
        for i in coloumn.values:
            if len(i) > max_len:
                max_len = len(i)

        if max_len % 2 == 1:
            max_len += 1

        padded = np.array([i + [0]*(max_len-len(i)) for i in coloumn.values])

        return pd.DataFrame(padded)

### Make model

In [7]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout, max_len):
        super(PositionalEncoding, self).__init__()

        # Info
        self.dropout = nn.Dropout(dropout)

        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1)
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-np.log(10000.0)) / dim_model) 

        # PE(pos, 2i) = = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)

        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)

        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding", pos_encoding)
    

    def forward(self, token_embedding):
        output = self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])
        return output

In [8]:
# Каждая заявка вносит свой вклад в приближение капитального ремонта
class ProcessRequestsNN(nn.Module):
    def __init__(self, num_features: int, num_add_features: int, hidden_dim: int = 64, emb_dim: int = 1024, modelBERT = None, num_requests: int = 19,
                 dropout: float = 0.1):
        super().__init__()
        self.num_features = num_features
        self.hidden_dim = hidden_dim
        self.embedding_size = emb_dim
        self.num_requests = num_requests
        self.output_conv = self.__compute_output() + num_add_features
        # Layers
        # Embedding
        if modelBERT is None:
            # make Bert Model for make embedding requests
            model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
            self.embed = model_class.from_pretrained(pretrained_weights)
            self.dimBERT = 768
        else:
            self.embed = modelBERT
            self.dimBERT = 768
        
        self.conv_1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5, stride=2, padding=4)
        self.max_pool_1 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        self.conv_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.max_pool_2 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        self.conv_3 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.max_pool_3 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        self.conv_4 = nn.Conv1d(in_channels=32, out_channels=1, kernel_size=3, stride=1, padding=1)
        self.max_pool_4 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        
        self.flatten = nn.Flatten()


        self.embed_after_conv = nn.Embedding(self.output_conv , self.embedding_size)
        self.linear_1 = nn.Linear(self.output_conv, int(self.embedding_size/2))
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(int(self.embedding_size/2), int(self.embedding_size/8))
        self.linear_3 = nn.Linear(int(self.embedding_size/8), self.num_requests)
        self.dropout = nn.Dropout(dropout)
        self.batch_norm = nn.BatchNorm1d(self.output_conv)

    
    def forward(self, in_data, in_data_2):
        attention_mask = torch.tensor(np.where(in_data != 0, 1, 0))
        #embed = model_class.from_pretrained(pretrained_weights)
        with torch.no_grad():
            out = self.embed(in_data, attention_mask=attention_mask)

        out = out[0].reshape(out[0].shape[0], out[0].shape[1]*out[0].shape[2]).unsqueeze(1)
        print("out.shape", out.shape)

        # Apply conv layer
        out = self.conv_1(out)
        out = self.max_pool_1(out)
        out = self.conv_2(out)
        out = self.max_pool_2(out)
        out = self.conv_3(out)
        out = self.max_pool_3(out)
        out = self.conv_4(out)
        out = self.max_pool_4(out)
        out = self.flatten(out)

        # concatenate date and name requests
        out = torch.concatenate(((out, in_data_2.permute(1,0))), dim=1)
        out = out.unsqueeze(0)
        # Apply Fully connected layer
        #out = self.batch_norm(out)
        #out = self.embed_after_conv(out.Lomnn)
        out = self.linear_1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.linear_3(out)

        return out
    

    def __compute_output(self):
        dim1 = 26882 
        dim_out = dim1
        return dim_out

In [9]:
buff_out = 0
buff_h = 0
buff_c = 0
buff_cat = 0

In [10]:
class ComputeResultNN(nn.Module):
    def __init__(self, num_features_req: int, num_features_cat: int = 14, num_output: int = 19,
                 emb_size: int = 128, hidden_dim: int = 64, dropout: float = 0.1, num_add_features: int = 10,
                 num_features_date_cup:int = 4):
        super().__init__()
        self.num_features_cat = num_features_cat
        self.emb_size = emb_size
        self.hid_dim = hidden_dim

        self.process_request = ProcessRequestsNN(num_features=num_features_req,
                                                 num_add_features=num_add_features)
        
        self.rnn = nn.LSTM(input_size = num_output,
                           hidden_size = self.hid_dim,
                           num_layers = 1, # may be 4 or 5 поставить
                           dropout=dropout,
                           bidirectional=True) #через lstm прогоняем все запросы получаем скрытое представление
        self.linear_1 = nn.Linear(num_features_cat + num_output, self.hid_dim)
        self.linear_2 = nn.Linear(self.hid_dim, 1)
        self.relu = nn.ReLU()
    

    def forward(self, in_data_cat, in_data_request, in_data_date):
        # proccess request
        out_req = self.process_request(in_data_request, in_data_date).squeeze(0)
        #out_req, (hid, cell) = self.rnn(out_req)
        # concatenate information
        multiply = torch.tensor([1 for i in range(19)])
        for i in out_req:
            multiply = torch.multiply(multiply, i)
        print(multiply.size())
        data_concat = torch.concatenate((multiply, in_data_cat), dim=0)

        # proccess specifications and out other model
        out = self.linear_1(data_concat.float())
        out = self.relu(out)
        out = self.linear_2(out)

        return out

### Train model

In [11]:
MAX_LEN = 70
NUM_FEATURES_REQ = MAX_LEN

In [12]:
# make model
model = ComputeResultNN(num_features_req=NUM_FEATURES_REQ,
                        num_features_cat=15,
                        num_output=19,
                        num_add_features=10,
                        num_features_date_cup=7)
loss_fn = nn.L1Loss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Anaconda\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropo

In [13]:
# получение даты и ее разности для запросов
def get_data_date(table):
    days_close = table["day_end"]

    time_perform_days = table["day_end"].to_numpy() - table["day_beg"]
    time_perform_month = table["month_end"].to_numpy() - table["month_beg"].to_numpy()
    time_perform_year = table["year_end"].to_numpy() - table["year_beg"].to_numpy()
    time_perform_hour = table["hour_end"].to_numpy() - table["hour_beg"].to_numpy()
    time_perform_minute = table["minute_end"].to_numpy() - table["minute_beg"].to_numpy()

    arr_times = [table["day_beg"].to_numpy(), table["month_beg"].to_numpy(), table["year_beg"].to_numpy(), 
                 table["hour_beg"].to_numpy(), table["minute_beg"].to_numpy(),time_perform_days,
                 time_perform_month, time_perform_year,
                 time_perform_hour,time_perform_minute]
    return torch.tensor(arr_times)


In [14]:
# вычисление ответа
def compute_y(table):
    d = {f'{i}': 0 for i in range(19)}
    d_exists = {f'{i}': 0 for i in range(19)}
    d_out = {}
    fact_beg_day = table["FACT_DATE_START"].dt.day.to_numpy()
    fact_beg_month = table["FACT_DATE_START"].dt.month.to_numpy()
    fact_beg_year  = table["FACT_DATE_START"].dt.year.to_numpy()
    plan_beg_day = table["PLAN_DATE_START"].dt.day.to_numpy()
    plan_beg_month = table["PLAN_DATE_START"].dt.month.to_numpy()
    plan_beg_year = table["PLAN_DATE_START"].dt.year.to_numpy()
    result = ((plan_beg_year - fact_beg_year) * 365 + (plan_beg_month - fact_beg_month) * 30 + (plan_beg_day - fact_beg_day))
    
    table = table.to_numpy()
    for count, name in enumerate(table):
        d[f"{name[0]}"] = (d[f"{name[0]}"] + result[count]) / 2
        d_exists[f"{name[0]}"] = 1
    for k in d.keys():
        if d_exists[k] == 1:
            d_out[k] = d[k]
    return d_out

In [15]:
dataset = CustomDataset(table_1, table_2, table_3, padded)
dataloader = DataLoader(dataset=dataset, batch_size=1)
X_batch = []
y_batch = []
N_EPOCHS = 5
BATCH_SIZE = 128
c = 0
train_epoch_loss = []
val_epoch_loss = []


for epoch in range(N_EPOCHS):
    train_loss = []
    val_loss = []
    for x,y in dataloader:
        for count, x_batch in enumerate(X_batch):
            y = compute_y(x_batch[2])
            print(y)
            for work_id in y.keys():
                data_request = x_batch[3] #данные по заявкам
                data_add_for_first_model = get_data_date(x_batch[1]) # данные времени по заявкам
                data_mlh = x_batch[0] # хар-ки дома
                number_work = float(work_id) # номер работы
                y_true = y[work_id]

                #in_data_cat - категориальные признаки + дата окончания последнего кап ремонта
                in_data_cat = torch.cat((torch.tensor(data_mlh), torch.tensor([int(work_id)])))

                # in_data_requests - заявки по данному уному
                in_data_requests = torch.tensor(data_request)
                # in_data_date - данные о времени заявок
                in_data_date = torch.tensor(data_add_for_first_model)
                print(in_data_requests.size(), in_data_date.size())

                preds = model(in_data_cat, in_data_requests, in_data_date)

                opt.zero_grad()
                loss = loss_fn(torch.tensor([y_true]), preds)
                loss.backward()
                opt.step()
                train_loss.append(loss)
        X_batch = []
        y_batch = []
    train_loss.append(sum(train_loss)/len(train_loss))

C:\Users\843E~1\AppData\Local\Temp/ipykernel_6188/6310117.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_3_unom['WORK_NAME'] = table_3_unom['WORK_NAME'].apply((lambda x: self.dict_all_works[x]))


[0, 0, 0, 0, 0, 0, 0, -12.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
{'7': -12.5}
torch.Size([193, 70]) torch.Size([10, 193])


C:\Users\843E~1\AppData\Local\Temp/ipykernel_6188/4294681698.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.tensor(arr_times)
C:\Users\843E~1\AppData\Local\Temp/ipykernel_6188/2775317569.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  in_data_date = torch.tensor(data_add_for_first_model)


out.shape torch.Size([193, 1, 53760])
torch.Size([19])
3
data_concat.size() torch.Size([34])
4
[-2.0, -20.5, 0, 0, -125.5, 0, 0, -12.5, -2.0, -17.0, 0, 0, -20.5, 0.5, -2.0, 43.0, 0, -20.5, 0]
{'0': -2.0, '1': -20.5, '4': -125.5, '8': -2.0, '9': -17.0, '12': -20.5, '13': 0.5, '14': -2.0, '15': 43.0, '17': -20.5}
torch.Size([141, 70]) torch.Size([10, 141])
out.shape torch.Size([141, 1, 53760])
torch.Size([19])
3
data_concat.size() torch.Size([34])
4
torch.Size([141, 70]) torch.Size([10, 141])
out.shape torch.Size([141, 1, 53760])
torch.Size([19])
3
data_concat.size() torch.Size([34])
4
torch.Size([141, 70]) torch.Size([10, 141])
out.shape torch.Size([141, 1, 53760])
torch.Size([19])
3
data_concat.size() torch.Size([34])
4


### Save/Load model

In [ ]:
checkpoint_path = "predict_dev.pth"

In [ ]:
def save_checkpoint(checkpoint_path, model, optimizer):
    # state_dict: a Python dictionary object that:
    # - for a model, maps each layer to its parameter tensor;
    # - for an optimizer, contains info about the optimizer’s states and hyperparameters used.
    state = {
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [ ]:
# create a new model
save_checkpoint("predict_dev.pth", model, opt)
model = model = ComputeResultNN(num_features_req=NUM_FEATURES_REQ,
                        num_features_cat=14,
                        num_output=19,
                        num_add_features=10,
                        num_features_date_cup=7)
opt = torch.optim.Adam(model.parameters(), lr=0.001, momentum=0.9)
# load from the final checkpoint
load_checkpoint('<CHECKPOINT NAME>', model, opt)

### Use model

In [ ]:
def get_padded(table_2, maxl_len=70):
    #BERT tokenizer
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

    # удаление нахер вссего NaN дерьма
    for count, elem in enumerate(table_2["Наименование"].isna()):
        if elem:
            table_2 = table_2.drop(labels=[count], axis=0)
    table_2["Наименование"].isna().sum()

    # apply tokenizator
    df = table_2["Наименование"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

    # find max_len
    max_len = 0
    for i in df.values:
        if len(i) > max_len:
            max_len = len(i)

    if max_len % 2 == 1:
        max_len += 1

    # add <pad> symbol
    padded = np.array([i + [0]*(max_len-len(i)) for i in df.values])

In [ ]:
#in_data_cat - категориальные признаки + дата окончания последнего кап ремонта (14 штук)
# in_data_requests - заявки по данному уному (заявки по уному в виде таблицы)
# in_data_date - данные о времени заявок (3 т)
def get_result(table_1, table_2, table_3, model):
    model.train(False)
    result = {}
    X_batch = []
    y_batch = []
    padded = get_padded(table_2)
    dataset = CustomDataset(table_1, table_2, table_3, padded)
    dataloader = DataLoader(dataset=dataset, batch_size=1)
    for x,_ in dataloader:
        for count, x_batch in enumerate(X_batch):
            y = compute_y(x_batch[2])
            print(y)
            for work_id in y.keys():
                data_request = x_batch[3] #данные по заявкам
                data_add_for_first_model = get_data_date(x_batch[1]) # данные времени по заявкам
                data_mlh = x_batch[0] # хар-ки дома
                number_work = float(work_id) # номер работы
                y_true = y[work_id]

                #in_data_cat - категориальные признаки + дата окончания последнего кап ремонта
                in_data_cat = torch.cat((torch.tensor(data_mlh), torch.tensor([int(work_id)])))

                # in_data_requests - заявки по данному уному
                in_data_requests = torch.tensor(data_request)
                # in_data_date - данные о времени заявок
                in_data_date = torch.tensor(data_add_for_first_model)

                preds = model(in_data_cat, in_data_requests, in_data_date)

                result[work_id] = preds
    return result

In [ ]:
def additional_training(table_1, table_2, table_3, opt, model):
    model.train(True)
    loss_fn = nn.L1Loss()
    padded = get_padded(table_2)
    dataset = CustomDataset(table_1, table_2, table_3, padded)
    dataloader = DataLoader(dataset=dataset, batch_size=1)
    for x,_ in dataloader:
        for count, x_batch in enumerate(X_batch):
            y = compute_y(x_batch[2])
            print(y)
            for work_id in y.keys():
                data_request = x_batch[3] #данные по заявкам
                data_add_for_first_model = get_data_date(x_batch[1]) # данные времени по заявкам
                data_mlh = x_batch[0] # хар-ки дома
                number_work = float(work_id) # номер работы
                y_true = y[work_id]

                #in_data_cat - категориальные признаки + дата окончания последнего кап ремонта
                in_data_cat = torch.cat((torch.tensor(data_mlh), torch.tensor([int(work_id)])))

                # in_data_requests - заявки по данному уному
                in_data_requests = torch.tensor(data_request)
                # in_data_date - данные о времени заявок
                in_data_date = torch.tensor(data_add_for_first_model)

                preds = model(in_data_cat, in_data_requests, in_data_date)

                opt.zero_grad()
                loss = loss_fn(torch.tensor(y_true), preds)
                loss.backward()
                opt.step()
    model.train(False)
    return model, opt